# Classify as above 50,000 or below 50,000

In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf

C:\Users\fq1228hj\AppData\Local\Continuum\anaconda3\envs\tfdeeplearning\lib\site-packages\tensorflow\python\framework\dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\fq1228hj\AppData\Local\Continuum\anaconda3\envs\tfdeeplearning\lib\site-packages\tensorflow\python\framework\dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\fq1228hj\AppData\Local\Continuum\anaconda3\envs\tfdeeplearning\lib\site-packages\tensorflow\python\framework\dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = n

In [2]:
data = pd.read_csv('census_data.csv')

In [3]:
data['income_bracket'][32560]

' >50K'

In [4]:
data.tail()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
32556,27,Private,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K
32560,52,Self-emp-inc,HS-grad,9,Married-civ-spouse,Exec-managerial,Wife,White,Female,15024,0,40,United-States,>50K


In [5]:
data['income_bracket'] = data['income_bracket'].apply(lambda x : 0 if (x == ' <=50K') else 1,) 

In [6]:
from sklearn.model_selection import train_test_split

In [13]:
X = data.drop('income_bracket',axis=1)
y = data['income_bracket']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [15]:
age = tf.feature_column.numeric_column("age")
education_num = tf.feature_column.numeric_column("education_num")
capital_gain = tf.feature_column.numeric_column("capital_gain")
capital_loss = tf.feature_column.numeric_column("capital_loss")
hours_per_week = tf.feature_column.numeric_column("hours_per_week")
gender = tf.feature_column.categorical_column_with_vocabulary_list("gender", ["Female", "Male"])
occupation = tf.feature_column.categorical_column_with_hash_bucket("occupation", hash_bucket_size=1000)
marital_status = tf.feature_column.categorical_column_with_hash_bucket("marital_status", hash_bucket_size=1000)
relationship = tf.feature_column.categorical_column_with_hash_bucket("relationship", hash_bucket_size=1000)
education = tf.feature_column.categorical_column_with_hash_bucket("education", hash_bucket_size=1000)
workclass = tf.feature_column.categorical_column_with_hash_bucket("workclass", hash_bucket_size=1000)
native_country = tf.feature_column.categorical_column_with_hash_bucket("native_country", hash_bucket_size=1000)

In [16]:
feat_cols = [gender,occupation,marital_status,relationship,education,workclass,native_country,
            age,education_num,capital_gain,capital_loss,hours_per_week]

In [17]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=100,shuffle=True,num_epochs=None)

In [18]:
model = tf.estimator.LinearClassifier(feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_summary_steps': 100, '_log_step_count_steps': 100, '_session_config': None, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_secs': 600, '_tf_random_seed': 1, '_model_dir': 'C:\\Users\\fq1228hj\\AppData\\Local\\Temp\\tmpb_cvgb9v', '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5}


In [19]:
model.train(input_fn=input_func,steps=5000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\fq1228hj\AppData\Local\Temp\tmpb_cvgb9v\model.ckpt.
INFO:tensorflow:step = 1, loss = 69.31474
INFO:tensorflow:global_step/sec: 232.09
INFO:tensorflow:step = 101, loss = 51.52557 (0.431 sec)
INFO:tensorflow:global_step/sec: 276.487
INFO:tensorflow:step = 201, loss = 96.91804 (0.366 sec)
INFO:tensorflow:global_step/sec: 239.739
INFO:tensorflow:step = 301, loss = 38.013866 (0.413 sec)
INFO:tensorflow:global_step/sec: 210.6
INFO:tensorflow:step = 401, loss = 68.41817 (0.475 sec)
INFO:tensorflow:global_step/sec: 221.783
INFO:tensorflow:step = 501, loss = 135.93227 (0.451 sec)
INFO:tensorflow:global_step/sec: 227.349
INFO:tensorflow:step = 601, loss = 123.10803 (0.440 sec)
INFO:tensorflow:global_step/sec: 243.993
INFO:tensorflow:step = 701, loss = 67.04837 (0.410 sec)
INFO:tensorflow:global_step/sec: 289.798
INFO:tensorflow:step = 801, loss = 302.22717 (0.345 sec)
INFO:tensorflow:global_step/se

In [21]:
pred_input_func = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=len(X_test),num_epochs=1,shuffle=False)

In [35]:
pred = model.predict(input_fn=pred_input_func)

In [36]:
pred1 = list(pred)

INFO:tensorflow:Restoring parameters from C:\Users\fq1228hj\AppData\Local\Temp\tmpb_cvgb9v\model.ckpt-5000


In [39]:
pred1[1]['class_ids'][0]

0

In [42]:
final_pred = []
for item in pred1:
    final_pred.append(item['class_ids'][0])

In [45]:
from sklearn.metrics import classification_report, confusion_matrix

In [49]:
print(confusion_matrix(y_test,final_pred))
print('\n')
print(classification_report(y_test,final_pred))

[[6423 1013]
 [ 668 1665]]


              precision    recall  f1-score   support

           0       0.91      0.86      0.88      7436
           1       0.62      0.71      0.66      2333

    accuracy                           0.83      9769
   macro avg       0.76      0.79      0.77      9769
weighted avg       0.84      0.83      0.83      9769

